# Практика (очистка данных)

In [95]:
import numpy as np
import pandas as pd
import seaborn as sns
diabetes_data = pd.read_csv('diabetes_data.csv', sep=',') # данные по диабету

# Небольшой разведывательный анализ 

#display(diabetes_data.tail())
#diabetes_data.info()
#diabetes_data.describe()
#display(diabetes_data['Gender'].describe())
#display(diabetes_data[diabetes_data['Age'] > 35].info()) # 276 из 778 записей с людьми старше 35 лет 


# Поиск дубликатов 
copy_diabetes = diabetes_data.copy()
list_col = list(copy_diabetes.columns)

mask = copy_diabetes.duplicated(subset=list_col) # булевая таблица 
copy_diabetes[mask].shape[0] # 10 полных дубликатов
copy_diabetes = copy_diabetes.drop_duplicates(subset=list_col) # удалили дубликаты


# очистим от неинформативных данных
cols = list(copy_diabetes.columns)
list_drop = []

def low_information_feature(data):
    for col in data.columns:
        repeat = data[col].value_counts(normalize=True).max() # одинаковых больше 99% - удаляем
        unique = data[col].nunique() / data[col].count() # если уникальных больше 0.99 - удаляем
    
        if repeat > 0.99:
            list_drop.append(col)
            print(f'{repeat*100}% повторяющихся зачений')
        if unique > 0.99:
            list_drop.append(col)
            print(f'{unique*100}% уникальных значений')
    return list_drop  

low_information_feature(copy_diabetes) # ['Gender'] не информативен 
copy_diabetes = copy_diabetes.drop(list_drop, axis=1) # что бы удалить строку, можно вместо столбца передать индекс



# Заменим 0 на Nan
col_with_null = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
def return_nan(series):
    if series== 0:
        series == np.nan
    else:
        return series
    
for i in col_with_null:
    copy_diabetes[i] = copy_diabetes[i].apply(return_nan)

copy_diabetes['Insulin'].isnull().value_counts(normalize=True) # Или взяв среднее (среди 1 и 0) получим % пропусков 


# Удалим признаки, где более 30% пропусков 
thresh = copy_diabetes.shape[0]*0.7
copy_diabetes = copy_diabetes.dropna(thresh= thresh, axis= 1)
copy_diabetes.shape[1] # 8


# Удалим записи, где более 2х пропусков одновременно
thresh = copy_diabetes.shape[1] 
copy_diabetes = copy_diabetes.dropna(thresh= thresh-2, axis=0)
copy_diabetes.shape[0] # 761


# Заменим оставшиеся пропуски на медиану
mas = copy_diabetes.isnull().mean() 
col_for_fill = mas[mas>0].index # Glucose, BloodPressure, SkinThickness, BMI - опредили столбцы для заполнения

values={
    'Glucose': copy_diabetes['Glucose'].median(),
    'BloodPressure': copy_diabetes['BloodPressure'].median(),
    'SkinThickness': copy_diabetes['SkinThickness'].median(),
    'BMI': copy_diabetes['BMI'].median()
}
copy_diabetes.fillna(values)
display(copy_diabetes['SkinThickness'].mean())


# более быстрый способ 
null_data = copy_diabetes.isnull().sum() 
cols = null_data[null_data>0].index # названия столбцов 
for col in cols:
    copy_diabetes[col] = copy_diabetes[col].fillna(copy_diabetes[col].median())
print(copy_diabetes['SkinThickness'].mean())
# Второй способ выдаст другой результат, потому что с copy_diabetes уже были проведены заполнения 
# По итогу пропусков в таблице не будет



# Выбросы в признаке SkinThickness по методу Тьюки (межквантильный разброс)
def serch_emissions(data, feature, right=1.5, left=1.5, log_scale=False):
    if log_scale:
        feature = np.log(data[feature])
    else:
        data[feature] 
        
    quantile_25 = data[feature].quantile(0.25)
    quantile_75 = data[feature].quantile(0.75)
    iqr = quantile_75 - quantile_25
    
    low_bound = quantile_25 -(left*iqr) 
    upp_bound = quantile_75 +(right*iqr) 
    
    outliers = data[(data[feature] < low_bound)|(data[feature] > upp_bound)] # записи с выбросами 
    cleaned = data[(data[feature] > low_bound)|(data[feature] < upp_bound)]
    
    return outliers, cleaned

clen_data = serch_emissions(copy_diabetes,'SkinThickness')
#display(clen_data[0].shape[0]) # выбросов 87
#display(clen_data[1].shape[0]) # чистых данных 761



# Выбросы в признаке SkinThickness по методу Z-отклонений
def z_outliers(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        feature = np.log(data[feature])
    else:
        x = data[feature]
    
    mu = x.mean()
    sigma = x.std()
    
    left_bound = mu - sigma*left
    right_bound = mu + sigma*right
    
    qutlires = data[(x < left_bound)|(x > right_bound)]
    cleaned = data[(x > left_bound)|(x < right_bound)]
    
    return qutlires, cleaned

result = z_outliers(copy_diabetes, 'SkinThickness')
#display(f'число выбросов - {result[0].shape[0]}, число очищенных данных - {result[1].shape[0]}') # число выбросов - 4



# сравним число выбросов в класичеком методе с логарифмированным
def serch_outliers(data, feature, right=1.5, left=1.5, log_scale=False):
    if log_scale:
        data[feature] = np.log(data[feature])
    else:
        data[feature] 
        
    quantile_25 = data[feature].quantile(0.25)
    quantile_75 = data[feature].quantile(0.75)
    iqr = quantile_75 - quantile_25
    
    low_bound = quantile_25 -(left*iqr) 
    upp_bound = quantile_75 +(right*iqr) 
    
    outliers = data[(data[feature] < low_bound)|(data[feature] > upp_bound)] # записи с выбросами 
    cleaned = data[(data[feature] > low_bound)|(data[feature] < upp_bound)]
    
    return outliers, cleaned

clen_data_classik = serch_outliers(copy_diabetes,'DiabetesPedigreeFunction')
clen_data_log = serch_outliers(copy_diabetes,'DiabetesPedigreeFunction', log_scale=True)

print(clen_data_classik[0].shape[0],',',clen_data_log[0].shape[0]) 
# После логарифмирования выбросов обнаружено не было 

100.0% повторяющихся зачений


29.153419593345657

29.109067017082786
29 , 0
